In [1]:
pip install llama-index


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 929.3/929.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency

In [2]:
from pathlib import Path
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI
import json
from typing import Sequence, List

from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index import VectorStoreIndex
from llama_index import download_loader

import json
from typing import Sequence

from llama_index.tools import BaseTool, FunctionTool
import llama_index

In [5]:
import json
import time
import os
def save_chat(chat_input, chat_output):
    """
    save chat with timestamp
    """
    chat = {
    "input":chat_input,
    "output":chat_output
}
    os.makedirs("./chat_log", exist_ok=True)
    with open(os.path.join("./chat_log", f'chat_{time.time()}.json'), 'w') as f:
        json.dump(chat, f)

In [8]:
PagedCSVReader = download_loader("PagedCSVReader")

llama_index.set_global_handler("simple")

# def initialize_index():
#     global index
#     loader = PagedCSVReader(encoding="utf-8")
#     documents = loader.load_data(file=Path('./documents/grocery.csv'))
#     index = VectorStoreIndex.from_documents(documents)
#     index.storage_context.persist(persist_dir="./storage/")

PagedCSVReader = download_loader("PagedCSVReader")
loader = PagedCSVReader(encoding="utf-8")
documents = loader.load_data(file=Path('./documents/grocery.csv'))
index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist(persist_dir="./storage/")


[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## OpenAI Agent with Query Engine Tools

In [6]:
def get_cart_item_quantity(item_title):
    """Gets the current quantity in cart of a certain item.
    It returns a tuple of (current_quatity, item_id)
    """
    return 3, 26



def set_cart_item_quantity(item_id, new_quantity):
    """Sets the quantity in cart of a certain item"""
    return 200

In [9]:
model = "gpt-4-1106-preview"

# the query engine that retrieves items from the vector index
grocery_query_engine = index.as_query_engine(similarity_top_k=5, model=model)
query_engine_tools = [
    QueryEngineTool(
        query_engine=grocery_query_engine,
        metadata=ToolMetadata(
            name="get_grocery_items",
            description=(
                "Retrieves relavant grocery items that are available in the inventory."
            ),
        ),
    )]

get_cart_item_quantity_tool = FunctionTool.from_defaults(fn=get_cart_item_quantity)
set_cart_item_quantity_tool = FunctionTool.from_defaults(fn=set_cart_item_quantity)
tools = query_engine_tools + [get_cart_item_quantity_tool, set_cart_item_quantity_tool]
stand_alone_agent = OpenAIAgent.from_tools(
    tools,
    verbose=True,
    model=model,
    system_prompt="""You are a grocery assistant who suggests available grocery items to online grocery shoppers.
    You only suggest items that are in the inventory, and none others.
    If an item isn't relevant to the user's query, don't list it.
    Always answer in one sentence or less.
    When describing the grocery items, you only use the information you get from the tool.
    You suggest a maximum of 5 relevant items from the inventory.
    If it's at all possible that the user is seeking grocery suggestions, you interpret their query that way and use the tool to find suggestions from the inventory.
    You do not provide suggestions that aren't related to groceries.
    If the user asks something that clearly has nothing to do with groceries in any concievable way, ask them to "Kindly fuck off. That has nothing to do with groceries.".
    You also are able to update the quantity of an item in the cart."""
)


In [10]:
chat_input = "Suggest me gluten free snacks"
response = outer_agent.chat(chat_input)
save_chat(chat_input, response.response)

NameError: ignored

### Halloween party snack scenario

In [ ]:
# response = await agent.astream_chat("What snacks would be good for a halloween party?")
# response_gen = response.rsponse_gen

# async for token in response.async_response_gen():
#     print(token, end="")

In [ ]:
# response.sources[0]

In [ ]:
# for source in response.sources:
#     for node in source.raw_output.source_nodes:
#        print(node.node)
# response
# response.sources[0].raw_output.__dict__.keys()

In [ ]:
# num_source_nodes = len(response.sources[0].raw_output.source_nodes)
# nodes = [response.sources[0].raw_output.source_nodes[i] for i in range(num_source_nodes)]

In [ ]:
# def csv_line_text_to_dict(line_text):
#     line_text = line_text.lstrip('\ufeff')
#     lines = line_text.strip().split('\n')
#     data_dict = {line.split(':')[0].strip(): line.split(':')[1].strip() for line in lines}
#     return data_dict

# for node in nodes:
# #     print(node.__dict__.keys())
#     print(csv_line_text_to_dict(node.node.text))

In [ ]:
# response = await agent.astream_chat("Tell me more about the first one")
# response_gen = response.response_gen

# async for token in response.async_response_gen():
#     print(token, end="")

### Hiking Scenario

In [ ]:
# response = await agent.astream_chat("I'm going on a three day hiking trip through the smoky mountains. What foods should I pack?")
# response_gen = response.response_gen

# async for token in response.async_response_gen():
#     print(token, end="")

In [ ]:
# response = await agent.astream_chat("I'm going on a three day hiking trip through the smoky mountains. What food should I pack?")
# response_gen = response.response_gen

# async for token in response.async_response_gen():
#     print(token, end="")

In [11]:
# llm = OpenAI(model="gpt-4-1106-preview")
# agent = OpenAIAgent.from_tools(
#     [get_cart_item_quantity_tool, set_cart_item_quantity_tool], llm=llm, verbose=True
# )

recommendations_tools = [ # tools to be used by the recommendations_agent
    QueryEngineTool(
        query_engine=grocery_query_engine,
        metadata=ToolMetadata(
            name="get_grocery_items",
            description=(
                "Retrieves relavant grocery items that are available in the inventory."
            ),
        ),
    )]

recommendations_agent = OpenAIAgent.from_tools(
    recommendations_tools,
    verbose=True,
    model="gpt-4-1106-preview",
    system_prompt="""You are an agent that suggests grocery items available in the inventory based on the user's query.
    You only suggest items that are in the inventory, and none others.
    You try to suggest 3-5 items.
    You only recommend grocery items that suit the user's query.
    If it's at all possible that the user is seeking grocery suggestions, you interpret their query that way and use the tool to find suggestions from the inventory.
    When describing the grocery items, you only use the information you get from the tool.
    You do not provide suggestions that aren't related to groceries.
    """
)

manage_cart_tools = [get_cart_item_quantity_tool, set_cart_item_quantity_tool]

manage_cart_agent = OpenAIAgent.from_tools(
    manage_cart_tools,
    verbose=True,
    model="gpt-4-1106-preview",
    system_prompt="""You are an agent that is able to manage an online grocery shopper's shopping cart.
    You can get the current quantity of an item in the cart and/or update the quantity of an item in the cart.
    """
)

In [12]:
outer_agent_tools = [
    QueryEngineTool(
        query_engine=recommendations_agent,
        metadata=ToolMetadata(
            name="recommendations_agent", description="Agent that can give grocery item recommendations based on the user's query that are available in the inventory."
        ),
    ),
    QueryEngineTool(
        query_engine=manage_cart_agent,
        metadata=ToolMetadata(
            name="manage_cart_agent",
            description="Tool that can manage the quantity of items in the cart.",
        ),
    ),
]


outer_agent = OpenAIAgent.from_tools(
    outer_agent_tools,
    verbose=True,
    model="gpt-4-1106-preview",
    system_prompt="""You are a grocery assistant chatbot.
    You choose the correct tool to use based on the user's query.
    You should use a tool whenever possible.
    If the user mentions food (or anything similar to that), you assume they're talking about groceries.
    If the tools can't be used for the user's query, respond to the query directly without calling a tool.
    You don't give general food recommendations that you didn't get from your tools.
    """
)
# outer_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)


In [ ]:
outer_agent.chat("hi")

STARTING TURN 1
---------------

** Messages: **
system: You are a grocery assistant chatbot. 
    You choose the correct tool to use based on the user's query. 
    You should use a tool whenever possible.
    If the user mentions food (or anything similar to that), you assume they're talking about groceries.
    If the tools can't be used for the user's query, respond to the query directly without calling a tool.
    You don't give general food recommendations that you didn't get from your tools. 
    
user: hi
**************************************************
** Response: **
assistant: Hello! How can I assist you today?
**************************************************




AgentChatResponse(response='Hello! How can I assist you today?', sources=[], source_nodes=[])

In [26]:
food_related_needs = [
    "Healthy breakfast options for busy mornings",
    "Low-carb meal ideas for a keto diet",
    "Refreshing beverage choices for a hot summer day",
    "Nutritious lunchbox ideas for school children",
    "Quick and easy dinner recipes for a family of six",
    "Dairy-free dessert options for lactose-intolerant individuals",
    "High-protein snack ideas for bodybuilders",
    "Festive appetizers for a New Year's Eve party",
    "Romantic dinner ideas for a Valentine's Day celebration",
    "Hearty soup recipes for cold winter evenings"
]

In [ ]:
for chat_input in food_related_needs:
    # chat_input = "Snacks for children below 5 years"
    response = outer_agent.chat(chat_input)
    save_chat(chat_input, response.response)

In [ ]:
chat_input = "Add 2 of the spooky cookies to my cart"
response = outer_agent.chat(chat_input)
save_chat(chat_input, response.response)

In [ ]:
chat_input = "Great. I'm going on a big hike. What food will I need?"
response = outer_agent.chat(chat_input)
save_chat(chat_input, response.response)

STARTING TURN 1
---------------

** Messages: **
system: You are a grocery assistant chatbot. 
    You choose the correct tool to use based on the user's query. 
    If the tools can't be used for the user's query, respond to the query directly without calling a tool.
    
user: hi
assistant: Hello! How can I assist you today?
user: Give me halloween food recommendations
assistant: None
tool: Based on your query for "halloween food," I have some spooky suggestions for you. How about trying some Spooky Cookies and Haunted Chips? These Halloween-themed snacks will add a fun and festive touch to your Halloween celebrations. Enjoy the delicious flavors while embracing the spooky spirit of the season!
assistant: I recommend trying some Spooky Cookies and Haunted Chips for your Halloween celebrations. These Halloween-themed snacks will add a fun and festive touch to your festivities. Enjoy the delicious flavors while embracing the spooky spirit of the season!
user: Add 2 of the spooky cookie

AgentChatResponse(response='For your big hike, I recommend trying the "Hikers Delight" snack. It is a convenient and energy-packed option that will keep you fueled during your hike. Enjoy your hiking adventure!', sources=[ToolOutput(content='Based on your query for "food for hiking," I suggest trying the "Hikers Delight" snack. It is a great option for hikers and provides a convenient and energy-packed snack while on the trail. Enjoy your hiking adventure!', tool_name='recommendations_agent', raw_input={'input': <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x111df5dd0>>}, raw_output=Response(response='Based on your query for "food for hiking," I suggest trying the "Hikers Delight" snack. It is a great option for hikers and provides a convenient and energy-packed snack while on the trail. Enjoy your hiking adventure!', source_nodes=[NodeWithScore(node=TextNode(id_='4b393dc1-ff8b-43e3-bef2-88dca6703632', embedding=None, metadata={}, excluded_embed_metadat

In [ ]:
print(outer_agent.chat("I'm going on a big hike. What food will I need?"))

Auto Eval code

In [33]:
bot_response = "For grilling, I recommend getting some high-quality hot dogs like Hebrew National. They have a delicious flavor and are perfect for BBQs. \n\nAs for sides, Knorr rice sides are a convenient and tasty option. They come in a variety of flavors and can be easily prepared on the grill or stove. \n\nI hope these recommendations help make your summer BBQ a success!"

In [32]:
user_prompt = "I'm hosting a summer BBQ for my friends. Could you recommend some essential items for grilling and sides?"

In [ ]:
# This code is for v1 of the openai package: pypi.org/project/openai
from openai import OpenAI
client = OpenAI(api_key="OPENAI_API_KEY")
import json

In [61]:
files = os.listdir('/content/chat_log')

In [62]:
answer = []
for file in files:
    with open('/content/chat_log/' + file) as f:
        data = json.load(f)
    response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
        {
        "role": "system",
        "content": "Your job is to rate interactions between a Grocery Chatbot and Users on a scale of 1-5.
        Take into account the length, quality, and variability of the suggestions while evaluating the response. Only output the final score as a number."
        },
        {
        "role": "user",
        "content": f"User: {data['input']}\nBot: {data['output']}"
        },
        {
        "role": "assistant",
        "content": "1"
        }
    ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    answer.append(response.choices[0].message.content)

In [63]:
len(answer)

31

In [64]:
len(files)

31

In [66]:
answer_int = [int(val) for val in answer]

In [67]:
import numpy as np
arr = np.array(answer_int)

In [68]:
arr

array([5, 1, 3, 4, 3, 5, 4, 4, 5, 2, 5, 2, 4, 4, 5, 4, 4, 4, 3, 4, 4, 5,
       1, 4, 4, 1, 2, 1, 1, 5, 4])

In [69]:
np.mean(arr)

3.4516129032258065

In [70]:
np.std(arr)

1.3640241620558695

In [71]:
files[0]

'chat_1702071956.526527.json'

In [76]:
for file in files[2:]:
    with open('/content/chat_log/' + file) as f:
        data = json.load(f)
        break

In [77]:
data

{'input': 'Suggest me food items for a christmas dinner for 4',
 'output': "I apologize, but I don't have any specific suggestions for a Christmas dinner for 4 based on the available inventory. However, I can provide some general ideas for a festive meal:\n\n1. Roast Turkey: A classic centerpiece for Christmas dinner, a roast turkey is a delicious and traditional choice. You can find a whole turkey in the meat section of the grocery store.\n\n2. Mashed Potatoes: Creamy and comforting, mashed potatoes are a perfect side dish for Christmas dinner. Look for potatoes in the produce section.\n\n3. Green Beans: Fresh or frozen green beans can be steamed or sautéed as a nutritious and colorful addition to your meal. Check the vegetable section for green beans.\n\n4. Cranberry Sauce: Tangy and sweet cranberry sauce pairs well with turkey. You can find canned cranberry sauce in the condiments aisle.\n\n5. Dinner Rolls: Soft and fluffy dinner rolls are a must-have for any holiday meal. Look for 